In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from torch import nn, optim
import torch

from env import train_data_path, validation_data_path, models_dir
from models.dataloaders import LocationTimeWithoutGroupBusDataLoader
from models.models import SimplerMLP

In [2]:
all_train_data = pd.read_csv(
    train_data_path,
    index_col=False,
    parse_dates=['last_modified1', 'last_modified2'],
)

all_validation_data = pd.read_csv(
    validation_data_path,
    index_col=False,
    parse_dates=['last_modified1', 'last_modified2'],
)

In [3]:
train_data = all_train_data[all_train_data.group == 26129].copy().reset_index(drop=True)
validation_data = all_validation_data[all_validation_data.group == 26129].copy().reset_index(drop=True)

In [13]:
percent = train_data.shape[0] / all_train_data.shape[0]
percent

0.07040745638894137

In [14]:
percent = validation_data.shape[0] / all_validation_data.shape[0]
percent

0.07032967032967033

In [4]:
train_ds = LocationTimeWithoutGroupBusDataLoader(train_data)
val_ds = LocationTimeWithoutGroupBusDataLoader(validation_data)

In [5]:
train_data_loader = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=12)
val_data_loader = DataLoader(val_ds, batch_size=256, shuffle=True, num_workers=12)

In [6]:
model = SimplerMLP(input_size=15)

In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
save_path = models_dir.joinpath('simple_only_26129')
save_path.mkdir(parents=True, exist_ok=True)

In [10]:
import json
import time

saved_data = []
num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    model = model.to(device)

    running_loss = 0.0

    time_start = time.time()
    for data in train_data_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_function(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    training_loss = running_loss / len(train_data_loader)

    with torch.no_grad():
        model.eval()
        running_loss = 0.0

        for data in val_data_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)

            running_loss += loss.item()

        validating_loss = running_loss / len(val_data_loader)

    model = model.to(torch.device('cpu'))

    torch.save(optimizer.state_dict(), save_path.joinpath(f"{epoch}-optimizer.pt"))
    torch.save(model.state_dict(), save_path.joinpath(f"{epoch}-model.pt"))
    saved_data.append({
        "epoch": epoch,
        "training_loss": training_loss,
        "validation_loss": validating_loss,
    })

    time_end = time.time()
    time_elapsed = (time_end - time_start) / 60
    print(f'Epoch [{epoch + 1}/{num_epochs}], Training = {training_loss:.3f}, Validation = {validating_loss:.3f}, Time = {time_elapsed:.2f}')

with open(save_path.joinpath('loss.json'), 'w') as f:
    json.dump(saved_data, f)

print("Done")

Epoch [1/100], Training = 36.826, Validation = 15.663, Time = 1.00
Epoch [2/100], Training = 15.395, Validation = 14.668, Time = 1.01
Epoch [3/100], Training = 14.827, Validation = 16.007, Time = 1.00
Epoch [4/100], Training = 14.565, Validation = 14.352, Time = 1.00
Epoch [5/100], Training = 14.213, Validation = 14.859, Time = 1.00
Epoch [6/100], Training = 14.022, Validation = 15.338, Time = 1.00
Epoch [7/100], Training = 13.775, Validation = 14.088, Time = 1.00
Epoch [8/100], Training = 13.254, Validation = 13.648, Time = 1.00
Epoch [9/100], Training = 12.772, Validation = 12.634, Time = 1.00
Epoch [10/100], Training = 12.197, Validation = 13.656, Time = 1.03
Epoch [11/100], Training = 12.004, Validation = 11.423, Time = 1.03
Epoch [12/100], Training = 11.713, Validation = 11.649, Time = 1.03
Epoch [13/100], Training = 11.489, Validation = 11.874, Time = 1.08
Epoch [14/100], Training = 11.267, Validation = 12.090, Time = 1.04
Epoch [15/100], Training = 11.074, Validation = 11.436, T